##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Text generation with an RNN

### Import TensorFlow and other libraries

In [2]:
import tensorflow as tf

import numpy as np
import os
import time

### Read the data

In [3]:
# Read, then decode for py2 compat.
text = open(os.path.join('data', 'training.c'), 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 16455 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

int f(int a, int b, int c) {
	 return a - 58 - b - 74 + c * 40 - c * 34;
}
// <end>


int f(int a, int c) {
	 return a - 87 * a + 10 + c - 4 + c + 16;
}
// <end>


int f(int a, int c) {
	 return a - 95 * c - 99 + c + 9;
}
// <end>




In [5]:
# The unique characters in the file
vocab = sorted(set(text))

## Process the text

In [6]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

### The prediction task

In [7]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [8]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [9]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

### Create training batches

We used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [10]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build The Model

In [11]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [12]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [13]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           9472      
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 37)            37925     
Total params: 3,985,701
Trainable params: 3,985,701
Non-trainable params: 0
_________________________________________________________________


## Train the model

In [16]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [17]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [18]:
# Directory where the checkpoints will be saved
checkpoint_dir = '/Users/miller/Desktop/training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [19]:
EPOCHS=100

In [20]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
2/2 [==============================] - 2s 771ms/step - loss: 3.5805
Epoch 2/100
2/2 [==============================] - 1s 683ms/step - loss: 3.3514
Epoch 3/100
2/2 [==============================] - 2s 826ms/step - loss: 8.2214
Epoch 4/100
2/2 [==============================] - 1s 661ms/step - loss: 3.3093
Epoch 5/100
2/2 [==============================] - 1s 687ms/step - loss: 3.3702
Epoch 6/100
2/2 [==============================] - 1s 663ms/step - loss: 3.3572
Epoch 7/100
2/2 [==============================] - 2s 788ms/step - loss: 3.3211
Epoch 8/100
2/2 [==============================] - 1s 692ms/step - loss: 3.2699
Epoch 9/100
2/2 [==============================] - 1s 733ms/step - loss: 3.2010
Epoch 10/100
2/2 [==============================] - 1s 734ms/step - loss: 3.1141
Epoch 11/100
2/2 [==============================] - 1s 720ms/step - loss: 2.9985
Epoch 12/100
2/2 [==============================] - 1s 697ms/step - loss: 2.8585
Epoch 13/100
2/2 [===================

## Generate text

### Restore the latest checkpoint

In [21]:
tf.train.latest_checkpoint(checkpoint_dir)

'/Users/miller/Desktop/training_checkpoints/ckpt_100'

In [22]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            9472      
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 37)             37925     
Total params: 3,985,701
Trainable params: 3,985,701
Non-trainable params: 0
_________________________________________________________________


### The prediction loop

In [24]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 100

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    state = ''
    while state != '<end>':
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])
        state += idx2char[predicted_id]
        if len(state) > 5:
            state = state[1:]


    return (start_string + ''.join(text_generated))